In [1]:
import pandas as pd


file = "en_fr"
df1 = pd.read_csv("/home/sutclw/Work/Preparation/Projects/translation/Notebooks/en_ru.csv")
df2 = pd.read_csv("/home/sutclw/Work/Preparation/Projects/translation/Notebooks/en_ru_2.csv")

#df = df1.append(df2)
df1.to_csv("test.csv")

In [2]:
df1

,Unnamed: 0,ru,en
0,1,Давайте что-нибудь попробуем!,Let's try something.
1,2,Мне пора идти спать.,I have to go to sleep.
2,3,Что ты делаешь?,What are you doing?
3,4,Что ты делаешь?,What do you make?
4,5,Что ты делаешь?,What're you doing?
...,...,...,...
656928,656929,Лифт всё ещё не работает?,Is the elevator still out of order?
656929,656930,"Я сказал тебе, чтобы ты оставался здесь.",I told you to stay here.
656930,656931,"Я сказал тебе, чтобы ты оставалась здесь.",I told you to stay here.
656931,656932,"Я сказал вам, чтобы вы оставались здесь.",I told you to stay here.


# Load Data

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

#tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ru-en")

#model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ru-en")

In [4]:
import torch
from translation.models.encoder import Encoder
from translation.models.decoder import Decoder
from translation.models.transformers import Transformer
from translation.training.littrainer import  LitTrainer
import numpy as np
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq
import pytorch_lightning as pl
from translation.models.transformers import Transformer
from translation.models.encoder import Encoder
from translation.models.decoder import Decoder
from pytorch_lightning import loggers as pl_loggers

def preprocess_function(batch,lang='en-ru'):
    l1 = lang.split('-')[0]
    l2 = lang.split('-')[1]
    model_inputs = tokenizer(
        batch[l1], max_length=max_input_length, truncation=True)

    # Set up the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(batch[l2], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


# Press the green button in the gutter to run the script.

    print('PyCharm')




torch.set_float32_matmul_precision('medium')
raw_dataset = load_dataset('csv',
        data_files='test.csv')
print(raw_dataset)
lang_pair = "en-ru"
l1 = lang_pair.split('-')[0]
l2 = lang_pair.split('-')[1]
lang_pair = "en-ru"
model_checkpoint = f"Helsinki-NLP/opus-mt-{lang_pair}"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
split = raw_dataset['train'].train_test_split(test_size=0.05, seed=42)
max_input_length = 128
max_target_length = 128

tokenized_datasets = split.map(
    preprocess_function,
    batched=True,
    remove_columns=split["train"].column_names,
)


data_collator = DataCollatorForSeq2Seq(tokenizer)
batch = data_collator([tokenized_datasets["train"][i] for i in range(0, 5)])
batch.keys()


train_loader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    batch_size=64,
    collate_fn=data_collator,
    num_workers=4, pin_memory=True,
)
valid_loader = DataLoader(
    tokenized_datasets["test"],
    batch_size=64,
    collate_fn=data_collator,
    num_workers=4, pin_memory=True
)



Using custom data configuration default-84210adc254c38ba


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

/home/sutclw/.local/lib/python3.10/site-packages/datasets/download/streaming_download_manager.py:776: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Dataset csv downloaded and prepared to /home/sutclw/.cache/huggingface/datasets/csv/default-84210adc254c38ba/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0.1', 'Unnamed: 0', 'ru', 'en'],
        num_rows: 656933
    })
})


  0%|          | 0/625 [00:00<?, ?ba/s]

/home/sutclw/anaconda3/envs/torch/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3581: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


  0%|          | 0/33 [00:00<?, ?ba/s]

In [5]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 624086
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 32847
    })
})

# Load Model

In [6]:
encoder = Encoder(vocab_size=tokenizer.vocab_size + 1,
                  max_len=128,
                  d_key=16,
                  d_model=128,
                  n_heads=8,
                  n_layers=4,
                  dropout_prob=0.1)
decoder = Decoder(vocab_size=tokenizer.vocab_size + 1,
                  max_len=128,
                  d_key=16,
                  d_model=128,
                  n_heads=8,
                  n_layers=4,
                  dropout_prob=0.1)
transformer = Transformer(encoder, decoder)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
encoder.to(device)
decoder.to(device)

Using encoder without a target.
cuda:0


Decoder(
  (embedding): Embedding(62519, 128)
  (pos_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_blocks): Sequential(
    (0): DecoderBlock(
      (linear1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (linear2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (linear3): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (mhatt1): MultiHeadAttention(
        (key): Linear(in_features=128, out_features=128, bias=True)
        (query): Linear(in_features=128, out_features=128, bias=True)
        (value): Linear(in_features=128, out_features=128, bias=True)
        (fc): Linear(in_features=128, out_features=128, bias=True)
      )
      (mhatt2): MultiHeadAttention(
        (key): Linear(in_features=128, out_features=128, bias=True)
        (query): Linear(in_features=128, out_features=128, bias=True)
        (value): Linear(in_features=128, out_features=128, bias=True)
        (fc): Linear(in_features

In [7]:
from torchinfo import summary
# summary(model, input_size=(16,512), dtypes=['torch.IntTensor'], device='cpu')
summary(transformer)

Layer (type:depth-idx)                        Param #
Transformer                                   --
├─Encoder: 1-1                                --
│    └─Embedding: 2-1                         8,002,432
│    └─PositionalEncoding: 2-2                --
│    │    └─Dropout: 3-1                      --
│    └─Sequential: 2-3                        --
│    │    └─EncoderBlock: 3-2                 198,272
│    │    └─EncoderBlock: 3-3                 198,272
│    │    └─EncoderBlock: 3-4                 198,272
│    │    └─EncoderBlock: 3-5                 198,272
│    └─LayerNorm: 2-4                         256
├─Decoder: 1-2                                --
│    └─Embedding: 2-5                         8,002,432
│    └─PositionalEncoding: 2-6                --
│    │    └─Dropout: 3-6                      --
│    └─Sequential: 2-7                        --
│    │    └─DecoderBlock: 3-7                 264,576
│    │    └─DecoderBlock: 3-8                 264,576
│    │    └─Decoder

In [8]:
def translate(input_sentence):
  # get encoder output first
    enc_input = tokenizer(input_sentence, return_tensors='pt').to(device)
    #print(device)
    enc_output = encoder(enc_input['input_ids'], enc_input['attention_mask'])

    # setup initial decoder input
    dec_input_ids = torch.tensor([[ int(tokenizer.vocab_size)]], device=device)
    dec_attn_mask = torch.ones_like(dec_input_ids, device=device)

  # now do the decoder loop
    for _ in range(32):
        dec_output = decoder(
            enc_output,
            dec_input_ids,
            enc_input['attention_mask'],
            dec_attn_mask,
        )

        # choose the best value (or sample)
        prediction_id = torch.argmax(dec_output[:, -1, :], axis=-1)

        # append to decoder input
        dec_input_ids = torch.hstack((dec_input_ids, prediction_id.view(1, 1)))

        # recreate mask
        dec_attn_mask = torch.ones_like(dec_input_ids)

        # exit when reach </s>
        if prediction_id == 0:
            break
  
    translation = tokenizer.decode(dec_input_ids[0, 1:])
    #print(translation)
    return(translation)

In [9]:
import torch.nn as nn
criterion = nn.CrossEntropyLoss(ignore_index=-100)
optimizer = torch.optim.Adam(transformer.parameters(),lr=1e-5)
torch.set_float32_matmul_precision('medium')

scaler = torch.cuda.amp.GradScaler()

epochs = 1

In [10]:
from datasets import load_metric

bleu_metric = load_metric("sacrebleu")
bert_metric = load_metric("bertscore")

/tmp/ipykernel_14172/3816762898.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric("sacrebleu")


# Training loop

In [11]:
from datetime import datetime
# A function to encapsulate the training loop
torch.set_float32_matmul_precision('medium')

scaler = torch.cuda.amp.GradScaler()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

def train(model, criterion, optimizer, train_loader, valid_loader, epochs):
    train_losses = np.zeros(epochs)
    test_losses = np.zeros(epochs)

    for it in range(epochs):
        model.train()
        t0 = datetime.now()
        train_loss = []
        for batch in train_loader:
            # move data to GPU (enc_input, enc_mask, translation)
            batch = {k: v.to(device) for k, v in batch.items()}

            # zero the parameter gradients
            optimizer.zero_grad()

            enc_input = batch['input_ids']
            enc_mask = batch['attention_mask']
            targets = batch['labels']

            # shift targets forwards to get decoder_input
            dec_input = targets.clone().detach()
            dec_input = torch.roll(dec_input, shifts=1, dims=1)
            dec_input[:, 0] = int(tokenizer.vocab_size)

            # also convert all -100 to pad token id
            dec_input = dec_input.masked_fill(
              dec_input == -100, tokenizer.pad_token_id)

            # make decoder input mask
            dec_mask = torch.ones_like(dec_input)
            dec_mask = dec_mask.masked_fill(dec_input == tokenizer.pad_token_id, 0)

            # Forward pass
            with torch.amp.autocast(device_type="cuda", dtype=torch.float16):
                outputs = model(enc_input, dec_input, enc_mask, dec_mask)
                loss = criterion(outputs.transpose(2, 1), targets)

            # Backward and optimize
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            
            scaler.update()
            #scheduler.step(optimizer)
            train_loss.append(loss.item())

        # Get train loss and test loss
        train_loss = np.mean(train_loss)

        model.eval()
        test_loss = []
        for batch in valid_loader:
            batch = {k: v.to(device) for k, v in batch.items()}

            enc_input = batch['input_ids']
            enc_mask = batch['attention_mask']
            targets = batch['labels']

            # shift targets forwards to get decoder_input
            dec_input = targets.clone().detach()
            dec_input = torch.roll(dec_input, shifts=1, dims=1)
            dec_input[:, 0] = int(tokenizer.vocab_size)

            # change -100s to regular padding
            dec_input = dec_input.masked_fill(
              dec_input == -100, tokenizer.pad_token_id)

            # make decoder input mask
            dec_mask = torch.ones_like(dec_input)
            dec_mask = dec_mask.masked_fill(dec_input == tokenizer.pad_token_id, 0)

            outputs = model(enc_input, dec_input, enc_mask, dec_mask)
            #with torch.amp.autocast(device_type=“cuda”, dtype=torch.float16):
            loss = criterion(outputs.transpose(2, 1), targets)
            test_loss.append(loss.item())
        test_loss = np.mean(test_loss)

        # Save losses
        train_losses[it] = train_loss
        test_losses[it] = test_loss

        dt = datetime.now() - t0
        translations = []
        targets = []
        bertscores = []
        bleuscores = []
        for  i in range(400):
            translations.append(translate(split['test'][i][l1])[:-4])
            targets.append(split['test'][i][l2])
            score =bert_metric.compute(predictions=[translations[i]], references=[targets[i]], lang="ru")['f1']
            bertscores.append(score)
            score =bleu_metric.compute(predictions=[translations[i]], references=[[targets[i]]])['score']
            bleuscores.append(score)
        lscheduler.step()
        print(f'Epoch {it+1}/{epochs}, Train Loss: {train_loss:.4f}, \
          Test Loss: {test_loss:.4f}, Duration: {dt}')
        print(f"Mean BERT {np.mean(bertscores)}, Mean Bleu: {np.mean(bleuscores)}" )

    return train_losses, test_losses

In [ ]:
train_losses, test_losses = train(
    transformer, criterion, optimizer, train_loader, valid_loader, epochs=2)

In [23]:
torch.save(transformer.state_dict(), "en_ru_transformer_16epochs.pt")

In [31]:
translations = []
targets = []
bertscores = []
bleuscores = []
for  i in range(1000):
    translations.append(translate(split['test'][i][l1])[:-4])
    targets.append(split['test'][i][l2])
    score =bert_metric.compute(predictions=[translations[i]], references=[targets[i]], lang="ru")['f1']
    bertscores.append(score)
    score =bleu_metric.compute(predictions=[translations[i]], references=[[targets[i]]])['score']
    bleuscores.append(score)

In [32]:
np.mean(bertscores)

0.9134581453204155

In [33]:
np.mean(bleuscores)

43.68201343735392

In [18]:
for i in range(0,len(translations)):
    print(translations[i])
    print(targets[i])
    print()

Том презирал Мэри.
Том презирал Мэри.

Он обещал, что сделает это.
Он обещал, что сделает это.

Ты похож на довольны.
Ты выглядишь довольной.

Как была твой урок?
Как прошло занятие?

Кто вызвал Анна?
Кто звонил Энн?

Займи его в сух!
Заточить его в темницу!

Чего ещё Том хочет?
Чего ещё Том хочет?

Том ушёл раньше.
Том ушёл раньше.

Том слишком молод, чтобы голосовать.
Тому рано голосовать.

Мы найдём решение, я уверен.
Мы найдём решение, я уверен.

Когда твой первый класс?
Когда у вас первый урок?

Я видел, как Том и Мэри держались руки.
Я видел, как Том с Мэри держались за руки.

Не говори мне, что я уже знаю.
Не говорите мне того, что я и так знаю.

Если бы женщины знали, сколько бы мы по нему много раз скучали, они бы ушли.
Если бы женщины знали, как мы по ним скучаем, они бы раньше уходили.

Откуда ты знаешь, что это неправда?
Откуда вы знаете, что это неправда?

Не говори с автобусом, пока он едет на автобусе.
Не разговаривайте с водителем автобуса во время движения.

Спокоядь Т

In [17]:
targets

['Том презирал Мэри.',
 'Он обещал, что сделает это.',
 'Ты выглядишь довольной.',
 'Как прошло занятие?',
 'Кто звонил Энн?',
 'Заточить его в темницу!',
 'Чего ещё Том хочет?',
 'Том ушёл раньше.',
 'Тому рано голосовать.',
 'Мы найдём решение, я уверен.',
 'Когда у вас первый урок?',
 'Я видел, как Том с Мэри держались за руки.',
 'Не говорите мне того, что я и так знаю.',
 'Если бы женщины знали, как мы по ним скучаем, они бы раньше уходили.',
 'Откуда вы знаете, что это неправда?',
 'Не разговаривайте с водителем автобуса во время движения.',
 'Молчание Тома меня удивило.',
 'Где ближайшая автобусная остановка?',
 'У меня ещё никогда не было такой сильной головной боли.',
 'Можно воспользоваться твоим туалетом?',
 'Том сказал, что надеется, что вы это сделаете.',
 'Том и Мэри в одной учебной группе.',
 'Сегодня я очень устала.',
 'Я хочу понять Тома.',
 'Нога опять затекла.',
 'Это я учу Тома французскому.',
 'Какие есть варианты?',
 'Том посмотрел наверх.',
 'Он не выглядит особе

In [5]:
!ls

/bin/bash: /home/sutclw/anaconda3/envs/torch/lib/libtinfo.so.6: no version information available (required by /bin/bash)
backuplogs		      logs	     translation
build			      main.py	     translation.egg-info
Encoder_Decoder_ru.ipynb      notebooks      Translation.ipynb
en_ru_transformer_8epochs.pt  setup.py	     Untitled.ipynb
Evaluation.ipynb	      test.csv	     WordStressTransformer.ipynb
Evaluation_pytorch.ipynb      Testing.ipynb


In [6]:
transformer.load_state_dict(torch.load('en_ru_transformer_8epochs.pt'))
transformer.eval()

Transformer(
  (encoder): Encoder(
    (embedding): Embedding(62519, 64)
    (pos_encoding): PositionalEncoding(
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer_blocks): Sequential(
      (0): EncoderBlock(
        (ln1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (ln2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (mha): MultiHeadAttention(
          (key): Linear(in_features=64, out_features=128, bias=True)
          (query): Linear(in_features=64, out_features=128, bias=True)
          (value): Linear(in_features=64, out_features=128, bias=True)
          (fc): Linear(in_features=128, out_features=64, bias=True)
        )
        (ann): Sequential(
          (0): Linear(in_features=64, out_features=256, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=256, out_features=64, bias=True)
          (3): Dropout(p=0.1, inplace=False)
        )
        (dropout): Dropout(p=0.1, inplace=False)
      

In [9]:
translations = []
targets = []
bertscores = []
bleuscores = []
for  i in range(100):
    translations.append(translate(split['test'][i][l1])[:-4])
    targets.append(split['test'][i][l2])
    score =bert_metric.compute(predictions=[translations[i]], references=[targets[i]], lang="ru")['f1']
    bertscores.append(score)
    score =bleu_metric.compute(predictions=[translations[i]], references=[[targets[i]]])['score']
    bleuscores.append(score)

In [10]:
translations

['Том презирал Мэри.',
 'Он обещал, что сделает это.',
 'Ты похож на довольны.',
 'Как была твой урок?',
 'Кто вызвал Анна?',
 'Займи его в сух!',
 'Чего ещё Том хочет?',
 'Том ушёл раньше.',
 'Том слишком молод, чтобы голосовать.',
 'Мы найдём решение, я уверен.',
 'Когда твой первый класс?',
 'Я видел, как Том и Мэри держались руки.',
 'Не говори мне, что я уже знаю.',
 'Если бы женщины знали, сколько бы мы по нему много раз скучали, они бы ушли.',
 'Откуда ты знаешь, что это неправда?',
 'Не говори с автобусом, пока он едет на автобусе.',
 'Спокоядь Тома удивил меня.',
 'Где находится ближайший автобус?',
 'У меня никогда раньше не было болит болит болит.',
 'Можно воспользоваться вашей туалетом?',
 'Том сказал, что надеется, что вы это сделаете.',
 'Том и Мэри изучают то же группе.',
 'Я сегодня очень устал.',
 'Я хочу понять Тома.',
 'Моя нога опять спит!',
 'Это я преподавал Тома французский.',
 'Какого варианты?',
 'Том посмотрел.',
 'Он не выглядит очень счастливым.',
 'Половор

In [25]:
translate("I like rice")


'私は<unk> が好きです<unk> </s>'

In [17]:
def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_h = trial.suggest_int("n_heads", 4, 8, 1)
    n_l = trial.suggest_int("n_layers", 2, 8, 1)
    #n_mod = trial.suggest_int("n_layers", 64, 128, 64)
   
    encoder = Encoder(vocab_size=tokenizer.vocab_size + 1,
                      max_len=512,
                      d_k=16,
                      d_model=128,
                      n_heads=n_h,
                      n_layers=n_l,
                      dropout_prob=0.1)
    decoder = Decoder(vocab_size=tokenizer.vocab_size + 1,
                      max_len=512,
                      d_k=16,
                      d_model=128,
                      n_heads=n_h,
                      n_layers=n_l,
                      dropout_prob=0.1)
    transformer = Transformer(encoder, decoder)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)
    encoder.to(device)
    decoder.to(device)
    return(transformer)

In [18]:
def translate(input_sentence, model):
  # get encoder output first
    enc_input = tokenizer(input_sentence, return_tensors='pt').to(device)
    #print(device)
    enc_output = model.encoder(enc_input['input_ids'], enc_input['attention_mask'])

    # setup initial decoder input
    dec_input_ids = torch.tensor([[ int(tokenizer.vocab_size)]], device=device)
    dec_attn_mask = torch.ones_like(dec_input_ids, device=device)

  # now do the decoder loop
    for _ in range(32):
        dec_output = model.decoder(
            enc_output,
            dec_input_ids,
            enc_input['attention_mask'],
            dec_attn_mask,
        )

        # choose the best value (or sample)
        prediction_id = torch.argmax(dec_output[:, -1, :], axis=-1)

        # append to decoder input
        dec_input_ids = torch.hstack((dec_input_ids, prediction_id.view(1, 1)))

        # recreate mask
        dec_attn_mask = torch.ones_like(dec_input_ids)

        # exit when reach </s>
        if prediction_id == 0:
            break
  
    translation = tokenizer.decode(dec_input_ids[0, 1:])
    #print(translation)
    return(translation)

In [20]:
from datetime import datetime
import torch.nn as nn

torch.set_float32_matmul_precision('medium')

scaler = torch.cuda.amp.GradScaler()

device='cuda'
def objective(trial,epochs=3):
    # Generate the model.
    train_losses = np.zeros(epochs)
    test_losses = np.zeros(epochs)
    model = define_model(trial).to(device)
    criterion = nn.CrossEntropyLoss(ignore_index=-100)
    lr = trial.suggest_float("lr", 1e-4, 1e-3, log=True)
    
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)
    for it in range(epochs):
        t0 = datetime.now()
        train_loss = []
        for batch in train_loader:
            # move data to GPU (enc_input, enc_mask, translation)
            batch = {k: v.to(device) for k, v in batch.items()}

            # zero the parameter gradients
            optimizer.zero_grad()

            enc_input = batch['input_ids']
            enc_mask = batch['attention_mask']
            targets = batch['labels']

            # shift targets forwards to get decoder_input
            dec_input = targets.clone().detach()
            dec_input = torch.roll(dec_input, shifts=1, dims=1)
            dec_input[:, 0] = int(tokenizer.vocab_size)

            # also convert all -100 to pad token id
            dec_input = dec_input.masked_fill(
              dec_input == -100, tokenizer.pad_token_id)

            # make decoder input mask
            dec_mask = torch.ones_like(dec_input)
            dec_mask = dec_mask.masked_fill(dec_input == tokenizer.pad_token_id, 0)

            # Forward pass
            with torch.amp.autocast(device_type="cuda", dtype=torch.float16):
                outputs = model(enc_input, dec_input, enc_mask, dec_mask)
                loss = criterion(outputs.transpose(2, 1), targets)

            # Backward and optimize
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            train_loss.append(loss.item())

        # Get train loss and test loss
        train_loss = np.mean(train_loss)

        model.eval()
        test_loss = []
        for batch in valid_loader:
            batch = {k: v.to(device) for k, v in batch.items()}

            enc_input = batch['input_ids']
            enc_mask = batch['attention_mask']
            targets = batch['labels']

            # shift targets forwards to get decoder_input
            dec_input = targets.clone().detach()
            dec_input = torch.roll(dec_input, shifts=1, dims=1)
            dec_input[:, 0] = int(tokenizer.vocab_size)

            # change -100s to regular padding
            dec_input = dec_input.masked_fill(
              dec_input == -100, tokenizer.pad_token_id)

            # make decoder input mask
            dec_mask = torch.ones_like(dec_input)
            dec_mask = dec_mask.masked_fill(dec_input == tokenizer.pad_token_id, 0)

            outputs = model(enc_input, dec_input, enc_mask, dec_mask)
            #with torch.amp.autocast(device_type=“cuda”, dtype=torch.float16):
            loss = criterion(outputs.transpose(2, 1), targets)
            test_loss.append(loss.item())
            
            
        translations=[]    
        targets = []
        bertscores = []
        bleuscores = []
        for  i in range(200):
            translations.append(translate(split['test'][i][l1], model)[:-4])
            targets.append(split['test'][i][l2])
            score =bert_metric.compute(predictions=[translations[i]], references=[targets[i]], lang="ru")['f1']
            bertscores.append(score)
            score =bleu_metric.compute(predictions=[translations[i]], references=[[targets[i]]])['score']
            bleuscores.append(score)
        print(f"Mean BERT {np.mean(bertscores)}, Mean Bleu: {np.mean(bleuscores)}" )
        test_loss = np.mean(test_loss)

        # Save losses
        train_losses[it] = train_loss
        test_losses[it] = test_loss

        dt = datetime.now() - t0


        print(f'Epoch {it+1}/{epochs}, Train Loss: {train_loss:.4f}, \
          Test Loss: {test_loss:.4f}, Duration: {dt}')
    return(np.mean(bertscores))


In [21]:
import optuna

In [22]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

[I 2023-03-01 20:19:43,969] A new study created in memory with name: no-name-268307ed-45a7-42d7-811c-7c4abee954ad


cuda:0


[W 2023-03-01 20:20:32,293] Trial 0 failed with parameters: {'n_heads': 7, 'n_layers': 3, 'lr': 0.00012237253778222962} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/sutclw/anaconda3/envs/torch/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_8878/1066740464.py", line 52, in objective
    scaler.step(optimizer)
  File "/home/sutclw/.local/lib/python3.10/site-packages/torch/cuda/amp/grad_scaler.py", line 341, in step
    retval = self._maybe_opt_step(optimizer, optimizer_state, *args, **kwargs)
  File "/home/sutclw/.local/lib/python3.10/site-packages/torch/cuda/amp/grad_scaler.py", line 287, in _maybe_opt_step
    if not sum(v.item() for v in optimizer_state["found_inf_per_device"].values()):
  File "/home/sutclw/.local/lib/python3.10/site-packages/torch/cuda/amp/grad_scaler.py", line 287, in <genexpr>
    if not sum(v.item() for v in optimizer

KeyboardInterrupt: 

In [60]:
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.7627686563134194
  Params: 
    n_heads: 5
    n_layers: 5
    lr: 0.00021099837151687136
